In [1]:
import pickle as pk
import requests

import bs4 as bs
from bs4 import XMLParsedAsHTMLWarning
import warnings

warnings.filterwarnings('ignore', category=XMLParsedAsHTMLWarning)

## Get Data

In [2]:
url = 'https://github.com/fadeeva/EDGAR_research/raw/refs/heads/master/data/reports.pickle'
response = requests.get(url)
reports = pk.loads(response.content)

len(reports.keys())

496

### Get Random Reports

In [3]:
import random

def get_rand_reports(tickers:list=[], number_of_reports:int=5)->dict:
    tickers = random.sample(list(reports.keys()), 5) if not tickers else tickers
    data = {ticker: random.sample(reports[ticker], number_of_reports) for ticker in tickers}

    return data

In [4]:
rand_reports = get_rand_reports()
rand_reports

{'NTAP': ['https://www.sec.gov/Archives/edgar/data/1002047/000095017023005545/ntap-20230127.htm',
  'https://www.sec.gov/Archives/edgar/data/1002047/000095017022025960/ntap-20221028.htm',
  'https://www.sec.gov/Archives/edgar/data/1002047/000095017022017904/ntap-20220729.htm',
  'https://www.sec.gov/Archives/edgar/data/1002047/000095017023045061/ntap-20230728.htm',
  'https://www.sec.gov/Archives/edgar/data/1002047/000095017021005014/ntap-20211029.htm'],
 'POOL': ['https://www.sec.gov/Archives/edgar/data/945841/000094584124000160/pool-20240930.htm',
  'https://www.sec.gov/Archives/edgar/data/945841/000094584121000178/pool-20210930.htm',
  'https://www.sec.gov/Archives/edgar/data/945841/000094584124000136/pool-20240630.htm',
  'https://www.sec.gov/Archives/edgar/data/945841/000094584123000135/pool-20230930.htm',
  'https://www.sec.gov/Archives/edgar/data/945841/000094584123000117/pool-20230630.htm'],
 'PLD': ['https://www.sec.gov/Archives/edgar/data/1045609/000156459022035526/pld-10q_20

In [5]:
aapl_reports = get_rand_reports(['AAPL'], 10)
aapl_reports

{'AAPL': ['https://www.sec.gov/Archives/edgar/data/320193/000032019321000056/aapl-20210327.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019324000081/aapl-20240629.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019324000069/aapl-20240330.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019323000077/aapl-20230701.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019321000065/aapl-20210626.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019324000006/aapl-20231230.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019325000008/aapl-20241228.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019322000070/aapl-20220625.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019325000057/aapl-20250329.htm',
  'https://www.sec.gov/Archives/edgar/data/320193/000032019320000062/aapl-20200627.htm']}

### Get Corpus

In [6]:
def get_report_corpus(url:str)->bs:
    headers={
        'User-Agent':'Dummy Company jane.dow@dummy.com',
        'Accept-Encoding':'gzip, deflate',
        'Host':'www.sec.gov'
    }
    response = requests.get(url, headers=headers, allow_redirects=True)

    return bs.BeautifulSoup(response.text, features='lxml')


In [11]:
corpus = get_report_corpus(rand_reports['NTAP'][0])

In [40]:
ticker, date = corpus.find('title').text.split('-')
ticker, date

('10', 'Q')

In [15]:
# corpus.find(id='f-5').text # Commission File Numbe

In [16]:
# corpus.find(id='f-8').text # I.R.S. Employer Identification No

## Consolidated Balance Sheet

## Management’s Discussion and Analysis of Financial Condition and Results of Operations

In [32]:
title_vars = [
    'ITEM 2. Management’s Discussion and Analysis of Financial Condition and Results of Operations'.lower(),
    'Management’s Discussion and Analysis of Financial Condition and Results of Operations'.lower(),
    'item 2', 'item 2.', 'item 2:'
]

In [38]:
def get_ids(corpus):
    mdaf = corpus.find_all('span')
    try:
        # tag = [tag for tag in mdaf if tag.string.lower() in title_vars][0]
        tag = [tag for tag in mdaf][0]
        return tag.find('a').get('href')[1:], tag.parent.parent.parent.next_sibling.find('a').get('href')[1:]
    except AttributeError:
        return 'no tag found'


In [39]:
get_ids(corpus)

'no tag found'

In [ ]:
reports['LHX'][:3]

['https://www.sec.gov/Archives/edgar/data/202058/000020205825000101/hrs-20250328.htm',
 'https://www.sec.gov/Archives/edgar/data/202058/000020205824000168/hrs-20240927.htm',
 'https://www.sec.gov/Archives/edgar/data/202058/000020205824000140/hrs-20240628.htm']

In [ ]:
corpus_ = get_report_corpus('https://www.sec.gov/Archives/edgar/data/202058/000020205824000140/hrs-20240627.htm')

In [ ]:
get_ids(corpus_)

IndexError: list index out of range

In [ ]:
mdaf = corpus.find_all('span')

for tag in mdaf:
    if tag.text.upper()=='ITEM 2.': #
        print(tag)
        break


<span style="color:#0000ff;font-family:'Helvetica',sans-serif;font-size:9pt;font-weight:400;line-height:100%;text-decoration:underline"><a href="#i056866be11a54295b8c21e0877b67331_67" style="color:#0000ff;font-family:'Helvetica',sans-serif;font-size:9pt;font-weight:400;line-height:100%;text-decoration:underline">Item 2.</a></span>
